Ćwiczenie:

- pobierz zbiór danych Rain in Australia (załączony do tej instrukcji)

- usuń kolumny mające więcej niż 30% brakujących wartości oraz kolumnę 'Risk-MM'

- dokonaj imputacji brakujących wartości zakładając że są one MCAR (Missing Completely At Random), tzn. zmienne kategoryczne należy zastąpić dominantą (najczęściej występującą w kolumnie wartością) a numeryczne medianą (wartością środkową w rosnąco posortowanej kolumnie)

- obetnij obserwacje odstające więcej niż 1.5 rozstępu ćwiartkowego

- znormalizuj (numeryczne) i zakoduj (kategoryczne) dane

- wykorzystując sklearn.linear_model.LogisticRegression, dla każdego z regionów naucz model przewidujący jutrzejsza pogodę 'RainTomorrow'

- sprawdź który z modeli najskuteczniej przewiduje pogodę w skali całego kraju (dla każdego klasyfikatora zbiór testowy powinien być próbkowany z podzbiorów testowych dla wszystkich regionów)

- sprawdź czy wybrany model miał najwyższe accuracy na własnym zbiorze testowym spośród wszystkich klasyfikatorów

- porównaj najlepszy klasyfikator z "klasyfikatorem" zawsze wybierającym dominującą wartość w zbiorze

- dla najlepszego klasyfikatora wyświetl confusion matrix

Przy wykonywaniu zadania należy pamiętać o stratyfikacji.

In [2]:
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype

Wczytuję zbiór danych

In [3]:
df = pd.read_csv("weatherAUS.csv")
df.shape

(142193, 24)

Usuwam kolumny z brakującymi danymi oraz kolumne RISK_MM

In [4]:
column_names = ['RISK_MM']
all_entries = len(df.index)
nan_values = df.isnull().sum().as_matrix()

for i in range(len(nan_values)):
    if (nan_values[i]/all_entries)>=0.3:
        column_names.append(df.columns[i])

for name in column_names:
    if name in df.columns:
        del df[name]

print("Deleted columns:", column_names)

Deleted columns: ['RISK_MM', 'Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm']


In [5]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,No,No


Implikuję brakujące wartości

In [6]:
nan_values = df.isnull().sum().as_matrix()
for i in range(len(nan_values)):
    if nan_values[i] != 0:
        if is_numeric_dtype(df[df.columns[i]].dtype):
            median = df[df.columns[i]].median()
            df[df.columns[i]].fillna(median, inplace=True)
        else:
            dominant = df.Location.mode().as_matrix()
            df[df.columns[i]].fillna(dominant[0], inplace=True)

Usuwam wartosci, ktore odstają więcej niż 1.5 rozstępu ćwiartkowego

In [8]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
lines_to_delete = []
for column in IQR.index:
    min_val = Q1[column] - 1.5 * IQR[column]
    max_val = Q1[column] + 1.5 * IQR[column]
    for i in range(len(df[column].values)):
        if min_val > df[column][i] or df[column][i] > max_val:
            lines_to_delete.append(i)
df.drop(lines_to_delete, inplace=True)
print("Usunięto", len(lines_to_delete), "linii.")

Usunięto 198555 linii.


Normalizuje dane numeryczne i koduje dane kategoryczne

In [9]:
column_list = []
for column in df.columns[1:]:
    if is_numeric_dtype(df[column].dtype):
        column_list.append(column)

df[column_list] = (df[column_list]-df[column_list].min())/(df[column_list].max()-df[column_list].min())

df['Date'] = df['Date'].astype('datetime64')
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
categorical_feature_mask = df.dtypes==object
categorical_cols = df.columns[categorical_feature_mask].tolist()
categorical_cols.remove('Location')
df[categorical_cols] = df[categorical_cols].apply(lambda col: le.fit_transform(col))

df.head(10)

,Date,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,0.709091,0.657895,0.75,14,0.813953,14,15,0.833333,0.857143,0.679487,0.269231,0.266129,0.341365,0.665455,0.677083,1,0
1,2008-12-02,Albury,0.490909,0.730263,0.00,15,0.813953,7,16,0.166667,0.785714,0.333333,0.307692,0.383065,0.369478,0.676364,0.763889,1,0
2,2008-12-03,Albury,0.690909,0.750000,0.00,16,0.860465,14,16,0.791667,0.928571,0.256410,0.371795,0.262097,0.405622,0.814545,0.725694,1,0
3,2008-12-04,Albury,0.556364,0.825658,0.00,5,0.348837,10,1,0.458333,0.321429,0.346154,0.192308,0.665323,0.570281,0.709091,0.840278,1,0
6,2008-12-07,Albury,0.741818,0.726974,0.00,14,0.953488,13,14,0.833333,0.857143,0.397436,0.230769,0.342742,0.385542,0.709091,0.774306,1,0
7,2008-12-08,Albury,0.501818,0.782895,0.00,14,0.604651,11,14,0.250000,0.607143,0.384615,0.230769,0.495968,0.461847,0.643636,0.805556,1,0
10,2008-12-11,Albury,0.709091,0.904605,0.00,4,0.488372,11,3,0.708333,0.214286,0.384615,0.269231,0.431452,0.405622,0.792727,0.920139,1,1
14,2008-12-16,Albury,0.578182,0.815789,0.00,15,0.953488,0,15,0.541667,0.785714,0.410256,0.346154,0.495968,0.469880,0.680000,0.829861,0,0
18,2008-12-20,Albury,0.578182,0.746711,0.00,11,0.395349,10,7,0.708333,0.214286,0.346154,0.320513,0.729839,0.742972,0.625455,0.725694,1,0
19,2008-12-21,Albury,0.640000,0.868421,0.00,9,0.348837,10,10,0.375000,0.321429,0.487179,0.346154,0.733871,0.650602,0.745455,0.868056,1,0


Tworze zbiory treningowe i testowe w zależności od regionu

In [44]:
from sklearn.model_selection import train_test_split
y = 'RainTomorrow'
X = df.columns.tolist()
for i in ['Date', 'Location', 'RainTomorrow']:
    X.remove(i)

training_set_per_location = {}
for location in set(df.Location):
    temp_df = df[df.Location == location]
    try:
        x_train, x_test, y_train, y_test = train_test_split(temp_df[X], temp_df[y],
                                                        test_size=0.2,
                                                        random_state=42, stratify=temp_df[y])
    except:
        print("Dla regionu", location, "bez stratyfikacji - za małe zrozniocowanie danych")
        x_train, x_test, y_train, y_test = train_test_split(temp_df[X], temp_df[y],
                                                        test_size=0.2,
                                                        random_state=42)
    training_set_per_location[location] = [x_train, x_test, y_train, y_test]


Dla regionu Katherine bez stratyfikacji - za małe zrozniocowanie danych


Uczę modele dla każdego regionu odpowiednio

In [11]:
from sklearn.linear_model import LogisticRegression

max = 0
best_loc = ''
for loc in training_set_per_location:
    lr = LogisticRegression()
    try:
        lr.fit(training_set_per_location[loc][0], training_set_per_location[loc][2])
        score = lr.score(training_set_per_location[loc][1], training_set_per_location[loc][3])
        print("Dla regionu:", loc, "model przewidujący pogodę ma skutecznosc", score)
        if loc != 'Katherine' and score > max: #Wynik dla Katherine jest nierelatywny
            max = score
            best_loc = loc
    except:
        print("Dla regionu:", loc, "nie można uzyc regresji - brak danych dla ktorejs z klas")

print("Najlepszy wynik dla:", best_loc, "-", max)

Dla regionu: Cairns model przewidujący pogodę ma skutecznosc 0.9347826086956522
Dla regionu: CoffsHarbour model przewidujący pogodę ma skutecznosc 0.8483606557377049
Dla regionu: SalmonGums model przewidujący pogodę ma skutecznosc 0.9140401146131805
Dla regionu: Walpole model przewidujący pogodę ma skutecznosc 0.8260869565217391
Dla regionu: Wollongong model przewidujący pogodę ma skutecznosc 0.9
Dla regionu: Dartmoor model przewidujący pogodę ma skutecznosc 0.8571428571428571
Dla regionu: Townsville model przewidujący pogodę ma skutecznosc 0.96875
Dla regionu: Witchcliffe model przewidujący pogodę ma skutecznosc 0.8765957446808511
Dla regionu: BadgerysCreek model przewidujący pogodę ma skutecznosc 0.888
Dla regionu: Darwin nie można uzyc regresji - brak danych dla ktorejs z klas
Dla regionu: Albany model przewidujący pogodę ma skutecznosc 0.8232758620689655
Dla regionu: MountGinini model przewidujący pogodę ma skutecznosc 0.9054726368159204
Dla regionu: Brisbane model przewidujący pog

Sprawdzam ile jest próbek w najmniejszym datasecie

In [13]:
number = 100
for loc in training_set_per_location:
    num_of_samples = len(training_set_per_location[loc][1].index)
    if num_of_samples < number:
        number = num_of_samples
number

46

Tworzę zbiór próbek testowych dla całego kraju, biorąc z każdego zbioru testowego po 46 próbek

In [37]:
df_kraj_x_test = training_set_per_location['Albany'][1].head(46).copy()
df_kraj_y_test = training_set_per_location['Albany'][3].head(46).copy()
for loc in training_set_per_location:
    if loc is not 'Albany':
        df_kraj_x_test.append(training_set_per_location[loc][1].head(46))
        df_kraj_y_test.append(training_set_per_location[loc][3].head(46))


In [38]:
max = 0
best_loc = ''
for loc in training_set_per_location:
    lr = LogisticRegression()
    try:
        lr.fit(training_set_per_location[loc][0], training_set_per_location[loc][2])
        score = lr.score(df_kraj_x_test, df_kraj_y_test)
        print("Dla kraju, model", loc, " przewidujący pogodę ma skutecznosc", score)
        if loc != 'Katherine' and score >= max: #Wynik dla Katherine jest nierelatywny
            max = score
            best_loc = loc
    except:
        print("Dla kraju, model", loc, "nie można uzyc regresji - brak danych dla ktorejs z klas")

print("Najlepszy wynik dla:", best_loc, "-", max)

Dla kraju, model Cairns  przewidujący pogodę ma skutecznosc 0.8695652173913043
Dla kraju, model CoffsHarbour  przewidujący pogodę ma skutecznosc 0.7391304347826086
Dla kraju, model SalmonGums  przewidujący pogodę ma skutecznosc 0.8043478260869565
Dla kraju, model Walpole  przewidujący pogodę ma skutecznosc 0.8043478260869565
Dla kraju, model Wollongong  przewidujący pogodę ma skutecznosc 0.8478260869565217
Dla kraju, model Dartmoor  przewidujący pogodę ma skutecznosc 0.8043478260869565
Dla kraju, model Townsville  przewidujący pogodę ma skutecznosc 0.8478260869565217
Dla kraju, model Witchcliffe  przewidujący pogodę ma skutecznosc 0.8043478260869565
Dla kraju, model BadgerysCreek  przewidujący pogodę ma skutecznosc 0.8913043478260869
Dla kraju, model Darwin nie można uzyc regresji - brak danych dla ktorejs z klas
Dla kraju, model Albany  przewidujący pogodę ma skutecznosc 0.8913043478260869
Dla kraju, model MountGinini  przewidujący pogodę ma skutecznosc 0.8478260869565217
Dla kraju, m

Najskuteczniejszy wynik na danych krajowych uzyskał model dla regionu BadgerysCreek z dokładnością 0.8913043478260869, ten sam model na lokalnych danych ma skuteczność wynoszącą 0.888, co nie jest najlepszym wynikiem dla modeli uczonych na danych regionalnych(Najlepszy wynik dla: AliceSprings - 0.971830985915493). Może być to spowodowane lepszym rozkładem danych krajowych nie danych regionalnych dla tego regionu. 

Confusion matrix dla regionu BadgerysCreek

In [45]:
from sklearn.metrics import confusion_matrix
lr = LogisticRegression()
lr.fit(training_set_per_location['BadgerysCreek'][0], training_set_per_location['BadgerysCreek'][2])
prediction = lr.predict(df_kraj_x_test)
confusion_matrix(df_kraj_y_test, prediction)

array([[40,  0],
       [ 5,  1]])

Na podstawie confusion matrix można stwierdzić, że model ma nierówną ilość danych dla poszczególnych klas (jest to najprawdopodobniej wada zbioru danych - znaczna przewaga jednej klasy nad drugą). Taki oparty na liniowej regresji może być wykorzystywany w wielu miejscach (nie tylko w predykcji pogody).